In [2]:
import os
import json
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models
import nltk

# Prepare Corpus

In [3]:
text = open("large_text.txt").read()

nltk.download('punkt_tab')
CORPUS = nltk.sent_tokenize(text)

CORPUS

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\lilyc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


['The sun was shining brightly in the clear blue sky, and a gentle breeze rustled the leaves of the tall trees.',
 'People were out enjoying the beautiful weather, some sitting in the park, others taking a leisurely stroll along the riverbank.',
 'Children were playing games, and laughter filled the air.',
 'As the day turned into evening, the temperature started to drop, and the sky transformed into a canvas of vibrant colors.',
 'Families gathered for picnics, and the smell of barbecues wafted through the air.',
 'It was a perfect day for a picnic by the lake.',
 'In the distance, you could hear the sound of live music coming from a local band, and people began to gather around the stage to enjoy the performance.',
 'The atmosphere was electric, and the music had everyone swaying to the beat.',
 'As the stars began to twinkle in the night sky, the crowd grew even larger, and the festivities continued well into the night.',
 'It was a day filled with joy, laughter, and memories that w